In [2]:
!pip install inspyred

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from random import Random
from time import time
from inspyred import ec, benchmarks

In [ ]:
class WorkoutRoutine(benchmarks.Benchmark):
    def __init__(self, userLevel, equipment, requestMuscles, time, userID, exercises):
        self.userLevel = userLevel
        self.time = time
        self.exercises = exercises
        length = time * 6 ##Tamaño del array se toma como valor el nivel beginner que como mucho son 6 ejercicios por hora
        benchmarks.Benchmark.__init__(self, length)
        self.equipment = equipment
        self.requestMuscles = requestMuscles
        self.userID = userID
        self.maximize = True
        self.bounder = ec.DiscreteBounder([1,3005])
    
    def generator(self, random):
        return random.choice(i for _ in range(3005))


    def evaluator(self, candidates):
        fitness = []
        for candidate in candidates:
            value = 100
            mins = self.total_mins(self,candidate)
            value = value * (mins / self.time * 60)
            value = value * self.used_equipment(self,candidate)
            value = value * self.trained_muscles(self,candidate)

            bonus = calculate_bonus(self,candidate)
            if value + bonus < 100:
                value += bonus
            else:
                value = 100


    def parseDificultad(dificultad):
        dificultad = dificultad.strip().lower()
        mapa_dificultad = {
            "beginner": 10,
            "novice": 12,
            "intermediate": 14,
            "advanced": 16,
            "expert": 18,
            "master": 20
        }
        return mapa_dificultad.get(dificultad, None)
    

    def total_mins(self,candidate):
        sum = 0
        for exercise in candidate:
            sum += self.parseDificultad(exercise) ##Añadir atributo de minutos
        return sum
    
    def used_equipment(self, candidate):
        used = []
        for exercise in candidate:
            if exercise.PrimaryEquipment != '' or exercise.PrimaryEquipment not in used:##Equipamiento##
                used.append(exercise.PrimaryEquipment)
            if exercise.SecondaryEquipment != '' or exercise.SecondaryEquipment not in used:##Equipamiento##
                used.append(exercise.SecondaryEquipment)
        return 0.1 * (len(used)/len(self.equipment)) + 0.9
    

    def trained_muscles(self, candidate):
        muscle_trained = []
        for exercise in candidate:
            if exercise.TargetMuscleGroup not in muscle_trained:##Equipamiento##
                muscle_trained.append(exercise.TargetMuscleGroup)
        
        return len(muscle_trained)/len(self.requestMuscles)
    
    def calculate_bonus(self, candidate):
        requested_regions = []
        total_bonus = 0
        for muscle in self.requestMuscles:
            ##Coger las diferentes bodyregion de los requestMuscles
            --
        for exercise in candidate:
            if exercise.BodyRegion in requested_regions and exercise.TargetMuscleGroup not in self.requestMuscles:
                total_bonus += 0.1
            elif exercise.TargetMuscleGroup not in self.requestMuscles:
                total_bonus += 0.05
        
        return total_bonus

NameError: name 'self' is not defined